In [8]:
from PIL import Image
import colorsys
import glob
import os
import numpy as np

def get_labels():
  return np.asarray([   (0, 0, 0),       # Background
                        (250, 149, 10),   # hood
                        (19, 98, 19),   # front door
                        (249,249,10), # rear door
                        (10,248,250), # frame
                        (149,7,149), # rear quarter panel
                        (11, 248, 7), # trunk lid
                        (20,19,249), # fender
                        (249,9,250)]#, # bumper
                        #(255, 255, 255)]    # rest of car
                        )

def rgb_to_hsl(rgb):
    r, g, b = rgb
    h, l, s = colorsys.rgb_to_hls(r / 255.0, g / 255.0, b / 255.0)
    return h, l, s

def hsl_to_rgb(hsl):
    h, l, s = hsl
    r, g, b = colorsys.hls_to_rgb(h, l, s)
    return int(r * 255), int(g * 255), int(b * 255)

def closest_color(target_color, color_list):
    target_hsl = rgb_to_hsl(target_color)
    closest_distance = float('inf')
    closest_color = hsl_to_rgb(target_hsl)

    for color in color_list:
        color_hsl = rgb_to_hsl(color)
        distance = sum((a - b) ** 2 for a, b in zip(target_hsl, color_hsl)) ** 0.5

        if distance < closest_distance and distance < 0.05:
            closest_distance = distance
            closest_color = color

    return closest_color

def change_similar_colors(image_path, color_list):
    image = image_path.copy()
    width, height = image.size

    for x in range(width):
        for y in range(height):
            pixel = image.getpixel((x, y))
            if pixel == (0,0,0):
              continue
            closest_color_pixel = closest_color(pixel, color_list)
            image.putpixel((x, y), (closest_color_pixel[0],closest_color_pixel[1],closest_color_pixel[2]))

    return image
#modded = change_similar_colors(target, get_labels())

In [11]:
file_list = os.path.join(os.path.join('data\\3seg', "*.*"))
file_list = glob.glob(file_list)
image_list = []
for file in file_list:
    tmp = Image.open(file)
    modded = change_similar_colors(tmp, get_labels())
    modded.save('data\\3segfix\\%s' % file.split("\\")[-1])
